Der alte code

In [34]:
import os
from bs4 import BeautifulSoup
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_ollama import OllamaLLM

# === XML-Dateien einlesen ===
xml_files = ["moduldb-pi2.xml", "Modulbook_pi2_de.xml"]
docs = []

for xml_file in xml_files:
    with open(xml_file, "r", encoding="ISO-8859-1") as f:
        xml_content = f.read()

    soup = BeautifulSoup(xml_content, "xml")
    modules = soup.find_all("moduleheader")

    for module in modules:
        title = module.find("title").text.strip() if module.find("title") else "Kein Titel"
        cid = module.find("cid").text.strip() if module.find("cid") else "Kein Kürzel"
        cp = module.find("cp").text.strip() if module.find("cp") else "?"
        convenor = module.find("convenor").text.strip() if module.find("convenor") else "?"

        types = module.find_all("type")
        ctypes = ", ".join(t.text for t in types) if types else "Unbekannt"

        content = f"""
Modul: {title}
Kürzel: {cid}
Leistungspunkte (ECTS): {cp}
Verantwortlich: {convenor}
Veranstaltungstyp(en): {ctypes}
"""
        docs.append(Document(page_content=content.strip()))

# Debug: Beispielinhalt zeigen
if docs:
    print("📄 Beispiel-Inhalt:\n", docs[0].page_content[:500])
else:
    print("⚠️ Keine Module gefunden.")

# === Dokumente splitten ===
splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)
chunks = splitter.split_documents(docs)

# === Embeddings & Chroma ===
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = Chroma.from_documents(chunks, embeddings)
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 1})

# === LLM über Ollama (Server) ===
llm = OllamaLLM(
    base_url="http://134.96.217.20:53100",
    model="llama3-70b",
    temperature=0.5,
    top_p=0.8,
    top_k=10,
    repeat_penalty=1.1,
    presence_penalty=1.2
)

# === Prompt ===
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "Hier ist ein Auszug aus Moduldaten:\n"
        "{context}\n\n"
        "Beantworte bitte diese Frage auf einfache Weise:\n"
        "{question}\n\n"
        "Antwort:"
    )
)

# === QA-Kette ===
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)

# === Chat starten ===
print("🤖 Chatbot gestartet! Tipp 'exit' oder 'quit' zum Beenden.\n")
while True:
    user_input = input("Du: ")
    if user_input.lower() in ["exit", "quit"]:
        print("🤖 Bot: Auf Wiedersehen!")
        break
    try:
        response = qa_chain.run(user_input)
        print("🤖 Bot:", response)
    except Exception as e:
        print(f"⚠️ Fehler: {e}")

📄 Beispiel-Inhalt:
 Modul: Bachelor-Abschlussarbeit
Kürzel: PIB-BT
Leistungspunkte (ECTS): 12
Verantwortlich: Studienleitung
Veranstaltungstyp(en): Unbekannt
🤖 Chatbot gestartet! Tipp 'exit' oder 'quit' zum Beenden.

🤖 Bot: Das Fach "Informatik 1" hat 5 ECTS-Leistungspunkte.
🤖 Bot: Auf Wiedersehen!


Der neue Code

In [2]:
## Einlesen der Daten und Definition Chatbot

# Import von langchain-Komponenten und externen Tools
from langchain.docstore.document import Document  # Datenstruktur für Dokumente
from langchain.prompts import PromptTemplate  # Für benutzerdefinierte Prompts
from langchain.chains import RetrievalQA  # Frage-Antwort-Kette auf Basis von Retrieval
from langchain_community.vectorstores import Chroma  # Vektor-Datenbank (Chroma)
from langchain.text_splitter import RecursiveCharacterTextSplitter  # Text-Splitter für Chunking
from langchain_huggingface import HuggingFaceEmbeddings  # Embedding-Modell von HuggingFace
from langchain_ollama import OllamaLLM  # Lokales Ollama-LLM
from bs4 import BeautifulSoup  # Für XML-Parsing

# === Inhalte.txt laden ===
docs = []  # Liste für Hauptdokumente
chunks = []  # Liste für vorbereitete Text-Chunks

# Inhalte.txt vollständig einlesen
with open("Inhalte.txt", "r", encoding="utf-8") as f:
    content = f.read()

# Aufteilen in einzelne Modulblöcke
blocks = content.split("============================================================")
for block in blocks:
    block = block.strip()
    if len(block) > 100:  # Filter für leere oder zu kleine Blöcke
        docs.append(Document(page_content=block))

print(f"📄 Datei 'Inhalte.txt' geladen. Module erkannt: {len(docs)}")

# === Chunking vorbereiten ===
splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=60)

# Manual chunking mit Modulnamen zur Orientierung
for doc in docs:
    first_line = doc.page_content.split("\n")[0]  # z.B. "Modul: Betriebssystemeinführung"
    vorlesung = first_line.split(":", 1)[1].strip().upper()
    for chunk_text in splitter.split_text(doc.page_content):
        full_chunk = f"{vorlesung}\n{chunk_text}\n{vorlesung}"  # Modulname oben & unten für Kontext
        chunks.append(Document(
            page_content=full_chunk,
            metadata={"modul": vorlesung.lower()}  # kann später für gezielte Filterung verwendet werden
        ))

# === XML-Dateien einlesen ===
xml_files = ["moduldb-pi2.xml", "modulbook_pi2_de.xml"]
docs = []  # XML-Daten überschreiben hier absichtlich vorherige docs-Liste

# XML parsen mit BeautifulSoup
for xml_file in xml_files:
    with open(xml_file, "r", encoding="ISO-8859-1") as f:
        xml_content = f.read()

    soup = BeautifulSoup(xml_content, "xml")
    modules = soup.find_all("moduleheader")  # Module sind unter <moduleheader> gelistet

    for module in modules:
        # Extraktion typischer Felder aus dem Modul
        title = module.find("title").text.strip() if module.find("title") else "Kein Titel"
        cid = module.find("cid").text.strip() if module.find("cid") else "Kein Kürzel"
        cp = module.find("cp").text.strip() if module.find("cp") else "?"
        convenor = module.find("convenor").text.strip() if module.find("convenor") else "?"

        types = module.find_all("type")
        ctypes = ", ".join(t.text for t in types) if types else "Unbekannt"

        content = f"""
Modul: {title}
Kürzel: {cid}
Leistungspunkte (ECTS): {cp}
Verantwortlich: {convenor}
Veranstaltungstyp(en): {ctypes}
"""
        docs.append(Document(page_content=content.strip()))  # Wieder als LangChain-Dokument speichern

print(f"{len(docs)} Dokumente aus xml Dateien extrahiert.")

# XML-Daten auch als Chunks splitten und hinzufügen
chunks.extend(splitter.split_documents(docs))

# === Vektordatenbank aufbauen ===
embeddings = HuggingFaceEmbeddings(model_name="multi-qa-mpnet-base-dot-v1")  # Hochwertiges QA-Embedding-Modell
db = Chroma.from_documents(chunks, embeddings)  # Chroma-Vektordatenbank aufbauen
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 7})  # Suche nach den 7 ähnlichsten Chunks

# === LLM starten ===
llm = OllamaLLM(
    base_url="http://134.96.217.20:53100",  # Ollama API-Adresse
    model="llama3-70b",  # Verwendetes Modell
    temperature=0.3  # Kreativität / Zufälligkeit der Antwort
)

# === Prompt für Frage-Antwort-System ===
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "Hier ist ein Auszug aus Modulbeschreibungen:\n"
        "{context}\n\n"
        "Beantworte die folgende Frage **nur auf Basis dieser Informationen**. "
        "Wenn die Antwort nicht im Text steht, sage: 'Nicht enthalten'.\n\n"
        "Frage: {question}\n\n"
        "Antwort:"
    )
)

# === QA-Kette starten ===
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",  # alle Chunks "stuffed" in ein Prompt
    chain_type_kwargs={"prompt": prompt}
)

## Run Chatbot
# === Interaktive Schleife ===
print("\n🤖 Chatbot gestartet! Tipp 'exit' oder 'quit' zum Beenden.\n")
while True:
    user_input = input("Du: ")
    if user_input.lower() in ["exit", "quit"]:
        print("🤖 Bot: Auf Wiedersehen!")
        break
    try:
        response = qa_chain.invoke({"query": user_input})  # Achtung: invoke braucht {"query": ...}
        print("🤖 Bot:", response["result"])  # Zeigt nur den Antworttext
    except Exception as e:
        print("⚠️ Fehler:", e)

## Test
# Einzelne Testabfrage (nicht interaktiv)
retriever.invoke("Literatur Betriebssystemeinführung?")  # Ruft relevante Chunks ab (kein LLM)


📄 Datei 'Inhalte.txt' geladen. Module erkannt: 30
121 Dokumente aus xml Dateien extrahiert.

🤖 Chatbot gestartet! Tipp 'exit' oder 'quit' zum Beenden.

🤖 Bot: Prof. Dr. Peter Birkner
🤖 Bot: Auf Wiedersehen!


[Document(metadata={'modul': 'betriebswirtschaftslehre'}, page_content='BETRIEBSWIRTSCHAFTSLEHRE\nLiteratur:\n- Jean-Paul Thommen, Ann-Kristin Achleitner: *Allgemeine Betriebswirtschaftslehre*, Springer\n- Thomas Straub: *Einführung in die Allgemeine Betriebswirtschaftslehre*, Pearson\n- Henner Schierenbeck, Claudia Wöhle: *Grundzüge der Betriebswirtschaftslehre*, Oldenbourg\n- Han\nBETRIEBSWIRTSCHAFTSLEHRE'),
 Document(metadata={'modul': 'betriebswirtschaftslehre'}, page_content='BETRIEBSWIRTSCHAFTSLEHRE\nLiteratur:\n- Jean-Paul Thommen, Ann-Kristin Achleitner: *Allgemeine Betriebswirtschaftslehre*, Springer\n- Thomas Straub: *Einführung in die Allgemeine Betriebswirtschaftslehre*, Pearson\n- Henner Schierenbeck, Claudia Wöhle: *Grundzüge der Betriebswirtschaftslehre*, Oldenbourg\n- Han\nBETRIEBSWIRTSCHAFTSLEHRE'),
 Document(metadata={'modul': 'betriebssysteme'}, page_content='BETRIEBSSYSTEME\nLiteratur:\n- J. Nehmer, P. Sturm: Systemsoftware – Grundlagen moderner Betriebssysteme, Pun